Function
-------
(Urban & tMin) - (Rural & tMin)

(Urban & tAvg) - (Rural & tAvg)


Stats
-----------------
 - June, July, August 2004-2013
 - Average summer (June to Aug daily temp)

In [2]:
import pickle
with open('climatecentral/climatecentralbase2.pickle', 'rb') as fout:
    sampleplaces = pickle.load(fout)

In [52]:
import numpy as np
import pandas as pd

outputdict = []

for s in sampleplaces.values():
    urbanuse = s['station'].get('urbanuse', None)
    if not urbanuse:
        print "skipping", s['usgsplacename']
        continue
    urbandf = pd.DataFrame(urbanuse['results'], columns=['measure', 'year', 'value'])
    
    ruraluse = s['station']['ruraluse']
    ruraldfs = []
    for r in ruraluse:
        tdf = pd.DataFrame(r['results'], columns=['measure', 'year', 'value'])
        tdf.set_index(['measure', 'year'])
        ruraldfs.append(tdf)
    df_concat = pd.concat(ruraldfs)
#     print ruraldfs

    ruralresult = pd.DataFrame(df_concat.groupby(['measure','year'])['value'].mean())

    
    mergeddf =  urbandf.merge(ruralresult.reset_index(),on=['measure','year'],how='left')
#     print mergeddf[['value_x', 'value_y']].sub(axis=1)
    mergeddf['uhi'] = mergeddf['value_x'] - mergeddf['value_y']
    outputdict.append({
            'placename': s['usgsplacename'],
            'uhitmin': mergeddf[mergeddf['measure']=='tmin']['uhi'].mean(),
            'uhitavg': mergeddf[mergeddf['measure']=='tavg']['uhi'].mean(),
            'uhitmax': mergeddf[mergeddf['measure']=='tmax']['uhi'].mean()
        })
#     print mergeddf[['value_x']].sub(mergeddf['value_y'], axis=0)
#     for year in range(2004, 2014):
#         for measure in ['tmin', 'tmax', 'tavg']


skipping Lubbock
skipping Eugene
[{'placename': u'Wichita', 'uhitmin': 1.0656663716073673, 'uhitmax': 1.7726580872480155, 'uhitavg': 1.4173166016585839}, {'placename': u'Springfield', 'uhitmin': -0.98528561745613885, 'uhitmax': 0.31563260805605753, 'uhitavg': -0.33624116325174069}, {'placename': u'Raleigh', 'uhitmin': 0.68458783257290601, 'uhitmax': -0.95819314230953123, 'uhitavg': -0.14538155344057202}, {'placename': u'Dallas', 'uhitmin': 2.3957038551325311, 'uhitmax': 1.3049480363703041, 'uhitavg': 1.8489620519962231}, {'placename': u'Lancaster', 'uhitmin': 1.5614481184158671, 'uhitmax': 11.259019200478999, 'uhitavg': 6.3844975601598133}, {'placename': u'Grand Rapids', 'uhitmin': 3.6822280514757408, 'uhitmax': 0.096103648806131051, 'uhitavg': 1.887962146437232}, {'placename': u'Modesto', 'uhitmin': 3.6200671621291214, 'uhitmax': 0.21808608717431924, 'uhitavg': 1.920507953937495}, {'placename': u'Houston', 'uhitmin': 0.57381411844773, 'uhitmax': 1.5585006437606908, 'uhitavg': 1.069527

In [53]:
import csv

with open('climatecentral/stationresults.csv', 'w') as fin:
    writer = csv.DictWriter(fin, fieldnames=['placename', 'uhitmin', 'uhitmax', 'uhitavg'])
    writer.writeheader()
    for row in outputdict:
        writer.writerow(row)

(-10854088.089812914, 4512349.452199243, -10810510.42127981, 4553634.740666818)
